<a href="https://colab.research.google.com/github/Clinda02/E-commerce_analysis/blob/main/CardGorilla_WebCrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 개요
* 카드고릴라 홈페이지 `카드` 탭의 전체 1,424 개 카드 정보를 수집하여 데이터 프레임으로 생성.  
* 데이터프레임을 `CardRorilla_check_card_test.csv`로 저장.
* 사이트 주소 : 카드고릴라 - https://www.card-gorilla.com  ([카드고릴라 robots.txt](https://www.card-gorilla.com/robots.txt))
---------------------------------------------------------
**카드고릴라 카드 탭**
  * 전체 카드 1,424 개의 카드 및 혜택 정보 (2025.01.11 기준)  
  * 카드 상품별로 페이지가 구분되어 있지 않아 페이지네이션 불가.  
  * [더보기] 버튼 클릭해 전체 카드 상품 로드 필요.

**데이터 프레임 구조**  
 * `카드명, 카드사, 이벤트, 발급중지, 연회비, 전월실적, 전월실적_텍스트, 카드종류, 혜택카테고리, 혜택상세` 10 개 칼럼으로 구성


## 카드고릴라 HTML 분석
### 1. 전체 카드 데이터 로드
1. for문으로 '더보기' 없을때 까지 클릭 (a clss="lst_more")

In [ ]:
# 라이브러리
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By  # 추가
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from tqdm import tqdm  # 진행 상태 표시 바

In [ ]:
# Selenium 웹 드라이버 서비스 설정
service = Service(executable_path=ChromeDriverManager().install())
url_CRD= 'https://www.card-gorilla.com/card?cate=CRD'  # 신용카드 상품
url_CHK = 'https://www.card-gorilla.com/card?cate=CHK' # 체크카드 상품

# Selenium 웹 드라이버 실행
driver = webdriver.Chrome(service = service)
driver.get(url_CRD)
wait = WebDriverWait(driver, 10) # 웹 페이지가 로드될 때까지, 최대 10초간 기다린다는 뜻

# 1 단계 : 전체 데이터 로드
click_cnt = 0
previous_html = driver.page_source  # 클릭 전 HTML

while True:
    try:
        # "더보기" 버튼 찾기
        next_button = driver.find_element(By.XPATH, '//*[@id="q-app"]/section/div[1]/section/div/div[2]/div[2]/a[1]')
        next_button.click()
        time.sleep(1)  # 데이터 로드 대기

        # 현재 페이지 HTML과 비교
        current_html = driver.page_source
        if previous_html == current_html:
            print("더 이상 새로운 데이터가 로드되지 않았습니다.")
            break
        previous_html = current_html  # 업데이트된 HTML 저장

        click_cnt += 1  # 클릭 횟수 저장
    except Exception as e:
        error_message = str(e).split("\n")[0]
        print(f"오류 발생: {error_message}")  # 예외 메시지 출력
        print(f"더보기 클릭 {click_cnt}회") # 클릭 횟수 출력
        print("더보기 버튼이 더 이상 없습니다.")
        break


# Selenium으로 로드된 페이지 소스 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

오류 발생: Message: element not interactable
더보기 클릭 99회
더보기 버튼이 더 이상 없습니다.


In [ ]:
soup

<html dir="ltr" lang="en-us" prefix="og: http://ogp.me/ns#"><head><base href="/"/><script src="https://teralog.techhub.co.kr/trk?la_tp=pv&amp;la_gc=CP4B3643229850&amp;la_uid=&amp;la_fid=d37a3106ad81451f482d8c7c803d4ded&amp;la_sid=&amp;la_infl=&amp;la_pa1=&amp;la_pa2=&amp;la_dccs=UTF-8&amp;la_r=118287375&amp;la_crsd_gc=&amp;la_sv=59647be&amp;la_dcrf=&amp;la_dcurl=https%3A%2F%2Fwww.card-gorilla.com%2Fcard%3Fcate%3DCRD&amp;la_dcttl=%EC%B9%B4%EB%93%9C%EA%B3%A0%EB%A6%B4%EB%9D%BC%20%3A%20%5B%EC%B9%B4%EB%93%9C%5D%20%EB%A7%9E%EC%B6%A4%EC%B9%B4%EB%93%9C%EA%B2%80%EC%83%89%2F%EC%B9%B4%EB%93%9C%EC%B6%94%EC%B2%9C%ED%85%8C%EC%8A%A4%ED%8A%B8&amp;la_ib=&amp;la_is=&amp;la_callback=_LA.CB"></script><script src="https://teralog.techhub.co.kr/ckie"></script><script src="https://sas.nsm-corp.com/sa-w.js?gc=CP4B3643229850&amp;rd=1736472723630"></script><script async="" src="https://www.clarity.ms/s/0.7.59/clarity.js"></script><script src="https://sas.nsm-corp.com/sa-w.js?gc=CP4B3643229850&amp;rd=17364727231

### 2. 각 카드별 상세 페이지로 유입
2. for문으로 개별 카드 페이지 들어가기 (ul class ="lst")<br>
2.1. `<a>` 태그 href="url" 정보로 이동. (class="b_view)

In [ ]:
# 카드 상품 개수
card_list = driver.find_elements(By.CSS_SELECTOR, 'ul.lst > li')
print(f"카드 상품 수 : {len(card_list)}")

카드 상품 수 : 998


In [ ]:
## 2 단계 : 각 카드의 상세 페이지 URL 추출
card_links = []
cards = soup.select('ul.lst > li a.b_view')  # 카드 상세보기 링크
for card in cards:
    link = card['href']
    full_url = f"https://www.card-gorilla.com{link}"  # 절대 URL로 변환
    card_links.append(full_url)

# 카드상품 수와 일치 확인
print(f"카드 페이지 수 : {len(card_links)}")

카드 페이지 수 : 998


In [ ]:
## 전체 데이터 합계
# 신용카드 : 998 건
# 체크카드 : 426 건

998 + 426

1424

### 3. 카드정보 1차 크롤링
1) div class="data_area"
    - **카드상품명** : div class="tit" / strong clss="card"
    - **카드사** : div class="tit" / p class="brand"
    - **이벤트** : a class=event_txt<br>
    - **발급중지** : a class=inactive > span > b <br>

In [ ]:
# card_links 중 필요한 부분만 추출
start_idx = 0  # 시작 인덱스
end_idx = 998  # 끝 인덱스
target_links = card_links[start_idx:end_idx]
print(f"총 {len(target_links)}개의 링크를 처리합니다.")

총 998개의 링크를 처리합니다.


In [ ]:
# 추출 데이터 저장할 리스트 및 딕셔너리
cards = []
card_data = {}

# 1) 카드 정보 가져오기
card_info1 = driver.find_elements(By.CSS_SELECTOR, 'div.data_area')

# 이벤트/ 카드상품명/ 카드사 데이터 추출
try :
  event_text = card_info1[0].find_element(By.CSS_SELECTOR, 'a.event_txt').text
except:
  event_text = "없음"
card_name = card_info1[0].find_element(By.CSS_SELECTOR, 'strong.card').text
brand_name = card_info1[0].find_element(By.CSS_SELECTOR, 'p.brand').text
try :
  stop_issued_element = card_info1[0].find_element(By.CSS_SELECTOR, 'div.app_btn > a.inactive > span > b')
  stop_issued = stop_issued_element.text  # 텍스트 추출
except:
  stop_issued = "없음"

print(event_text)
print(card_name)
print(brand_name)
print(stop_issued)

2) div class="bnf2" di[0]<br>
    - **연회비** : 국내전용, 해외겸용 (dd class="in_out")
    - **전월실적** : 없음, 20만원 등(dd)
    - **브랜드** : VISA, Mastercard 등 (dd class="c_brand")<br>

In [ ]:
# 2) 카드 조건 가져오기
card_info3 = driver.find_elements(By.CSS_SELECTOR, 'div.bnf2')

# 연회비/ 전월실적/ 카드 카테고리 데이터 추출
annual_fee = card_info2[0].find_element(By.CSS_SELECTOR, 'dd.in_out').text
performance = card_info2[0].find_element(By.XPATH, '//*[@id="q-app"]/section/div[1]/section/div/article[1]/div/div/div[3]/dl[2]/dd').text
performance_txt = card_info2[0].find_element(By.CSS_SELECTOR, 'dd >b').text
c_brand = card_info2[0].find_element(By.CSS_SELECTOR, 'dd.c_brand').text

print(annual_fee)
print(performance)
print(performance_txt)
print(c_brand)

국내전용 없음 해외겸용 없음
20만원 이상
20
VISAMastercard


3) 주요혜택 2차 크롤링 : article class="cmd_con benefit" 또는 div class="lst bene_area" dl[0]<br>
    - **혜택 카테고리** : 대중교통, 캐시백 (p class="txt1")  
    - **혜택 상세** : 할인 금액 및 할인율 (i)

In [ ]:
# 3) 카드 상세 내용 가져오기
card_info3 = driver.find_elements(By.CSS_SELECTOR, 'div.lst')

# 혜택 카테고리/ 상세 내용 데이터 추출
bnf_category= card_info3[0].find_element(By.CSS_SELECTOR, 'p').text
bnf_txt = card_info3[0].find_element(By.CSS_SELECTOR, 'i').text

print(bnf_category)
print(bnf_txt)

카페
[일상] 스타벅스, 커피빈 10% 할인


In [ ]:
# 데이터프레임 생성
df = pd.DataFrame(cards)
df.head()

,카드명,카드사,이벤트,발급중지,연회비,전월실적,전월실적_텍스트,카드종류,혜택카테고리,혜택상세
0,썸타는 우리 체크,우리카드,없음,없음,국내전용 없음 해외겸용 없음,25,25만원 이상,BCMastercard,수수료우대,[해외서비스] 해외이용수수료 면제
1,아이행복 체크카드,KB국민카드,없음,없음,해외겸용 없음,10,10만원 이상,Mastercard,아이행복,아이행복카드란?
2,아이행복체크카드(비씨),NH농협카드,없음,없음,해외겸용 없음,20,20만원 이상,UnionPay,온라인쇼핑,온라인쇼핑몰/소셜커머스 3~5% 청구할인
3,알뜰교통카드 S20 체크,신한카드,없음,없음,해외겸용 없음,20,20만원 이상,Mastercard,교통,전국 지하철/버스/택시 이용 시 10% 캐시백
4,에이스플러스체크카드 (벨리미키),SC제일은행,없음,없음,국내전용 없음 해외겸용 없음,10,10만원 이상,BCMastercard,영화,"영화 인터넷 예매시 1,500원 청구할인"


In [ ]:
## 3 단계 : 카드 페이지 크롤링 (전체)
# 전체 카드 정보를 저장할 빈 리스트
cards = []

for link in tqdm(target_links, desc="카드 데이터 수집 진행 중"):
  driver.get(link) # URL 이동

  # 페이지 로드 대기 (최대 10초)
  WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div.data_area')))

  # 하나의 카드 정보를 저장할 딕셔너리
  card_data = {}

  # 1) 카드 정보 가져오기
  card_info1 = driver.find_elements(By.CSS_SELECTOR, 'div.data_area')

  # 이벤트/ 카드상품명/ 카드사 데이터 추출
  try :
    event_text = card_info1[0].find_element(By.CSS_SELECTOR, 'a.event_txt').text
  except:
    event_text = "없음"
  card_name = card_info1[0].find_element(By.CSS_SELECTOR, 'strong.card').text
  brand_name = card_info1[0].find_element(By.CSS_SELECTOR, 'p.brand').text
  try :
    stop_issued_element = card_info1[0].find_element(By.CSS_SELECTOR, 'div.app_btn > a.inactive > span > b')
    stop_issued = stop_issued_element.text  # 텍스트 추출
  except:
    stop_issued = "없음"

  # 2) 카드 조건 가져오기
  card_info2 = driver.find_elements(By.CSS_SELECTOR, 'div.bnf2')

  # 연회비/ 전월실적/ 카드 카테고리 데이터 추출
  annual_fee = card_info2[0].find_element(By.CSS_SELECTOR, 'dd.in_out').text
  performance_txt = card_info2[0].find_element(By.XPATH, '//*[@id="q-app"]/section/div[1]/section/div/article[1]/div/div/div[3]/dl[2]/dd').text
  try:
    performance = card_info2[0].find_element(By.CSS_SELECTOR, 'dd >b').text
  except:
    performance = "없음"
  try:
    c_brand = card_info2[0].find_element(By.CSS_SELECTOR, 'dd.c_brand').text
  except:
    c_brand = "없음"

  # 3) 카드 상세 내용 가져오기
  card_info3 = driver.find_elements(By.CSS_SELECTOR, 'div.lst')

  # 혜택 카테고리/ 상세 내용 데이터 추출
  bnf_category= card_info3[0].find_element(By.CSS_SELECTOR, 'p').text
  bnf_txt = card_info3[0].find_element(By.CSS_SELECTOR, 'i').text

  # 텍스트 출력
  card_data['카드명'] = card_name
  card_data['카드사'] = brand_name
  card_data['이벤트'] = event_text
  card_data['발급중지'] = stop_issued
  card_data['연회비'] = annual_fee
  card_data['전월실적'] = performance
  card_data['전월실적_텍스트'] = performance_txt
  card_data['카드종류'] = c_brand
  card_data['혜택카테고리'] = bnf_category # 유의사항 행 제외 필요
  card_data['혜택상세'] = bnf_txt

  cards.append(card_data)

# 데이터프레임 생성
df = pd.DataFrame(cards)
# df.head()

# Excel 파일 저장
file_name= f"CardRorilla_check_card_test_{start_idx}_{end_idx}.xlsx"
# file_name= f"CardRorilla_credit_card_{start_idx}_{end_idx}.xlsx"
df.to_excel(file_name, index=True)
print(f"{file_name} 파일로 저장되었습니다")

# 드라이버 종료
# driver.quit()

카드 데이터 수집 진행 중: 100%|██████████| 998/998 [1:18:25<00:00,  4.72s/it]


CardRorilla_check_card_test_0_998.xlsx 파일로 저장되었습니다


In [ ]:
# 브라우저 세션 확인
try:
    print(driver.title)
except Exception as e:
    print("브라우저 세션이 닫혔습니다:", e)

브라우저 세션이 닫혔습니다: name 'driver' is not defined
